## Install the package dependencies before running this notebook

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import math

"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""

'\n    number of trajectories in each city\n    # austin --  train: 43041 test: 6325 \n    # miami -- train: 55029 test:7971\n    # pittsburgh -- train: 43544 test: 6361\n    # dearborn -- train: 24465 test: 3671\n    # washington-dc -- train: 25744 test: 3829\n    # palo-alto -- train:  11993 test:1686\n\n    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds\n    \n'

## Create a Torch.Dataset class for the training dataset

In [2]:
from glob import glob
import pickle
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

ROOT_PATH = "./"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", valid=False, normalized=False):
    f_in = ROOT_PATH + split + "/" + city + "_inputs"
    inputs = pickle.load(open(f_in, "rb"))
    inputs = np.asarray(inputs)
    
    outputs = None
    
    if split=="train":
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)
        
        if valid:
            idx = int(len(inputs) * .8)
            return inputs[:idx], inputs[idx:], outputs[:idx], outputs[idx:]

    return inputs, outputs

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform
        self.split = split
        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=False)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        
        if self.split == 'train':
            data = (self.inputs[idx], self.outputs[idx])
        else:
            data = (self.inputs[idx])
            
        if self.transform:
            data = self.transform(data)

        return data
    
class ValidationDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, inputs, outputs, transform=None):
        super(ValidationDataset, self).__init__()
        self.transform = transform
        self.inputs, self.outputs = inputs, outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        
        data = (self.inputs[idx], self.outputs[idx])
            
        if self.transform:
            data = self.transform(data)

        return data

# intialize a dataset
city = 'palo-alto' 
split = 'train'
train_dataset  = ArgoverseDataset(city = city, split = split)

## Create a DataLoader class for training

In [3]:
batch_sz = 32  # batch size 
train_loader = DataLoader(train_dataset, batch_size=batch_sz)

## Sample a batch of data and visualize 

In [4]:
import matplotlib.pyplot as plt
# import random

# def show_sample_batch(sample_batch):
#     """visualize the trajectory for a batch of samples"""
#     inp, out = sample_batch
#     batch_sz = inp.size(0)
#     agent_sz = inp.size(1)
    
#     fig, axs = plt.subplots(1, batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
#     fig.subplots_adjust(hspace = .5, wspace=.001)
#     axs = axs.ravel()   
#     for i in range(batch_sz):
#         axs[i].xaxis.set_ticks([])
#         axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
#         axs[i].scatter(inp[i,:,0], inp[i,:,1])
#         axs[i].scatter(out[i,:,0], out[i,:,1])

        
# for i_batch, sample_batch in enumerate(train_loader):
#     # inp[i] is a scene with 50 coordinates, input[i, j] is a coordinate
#     # gotta loop through each scene in the batch
#     inp, out = sample_batch # inp: (batch size, 50, 2), out: (batch size, 60, 2)
#     """
#     TODO:
#       implement your Deep learning model
#       implement training routine
#     """
#     show_sample_batch(sample_batch)
#     break

In [5]:
# from d2l.ai
def grad_clipping(net, theta):
    """Clip the gradient."""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm


class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, device, num_layers=1, dropout=0, bidirectional=False):
        super(RNN, self).__init__()
        self.device = device
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        if bidirectional:
            self.fc = nn.Linear(2 * hidden_size, output_size)
        else:
            self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.to(self.device)
        
        x = F.layer_norm(x, x.size())
        
        out, _ = self.rnn(x)
        
        out = self.fc(out)

        return out

import seaborn as sns
def train(net, n_epochs, train_loader, loss_fct, criterion, device, val_loader=None, valid=False):
    train_l= []
    val_l = []
    for epoch in range(n_epochs):
        ## training loop
        for i_batch, batch in enumerate(train_loader):
            # inp[i] is a scene with 50 coordinates, input[i, j] is a coordinate
            inp, out = batch
            inp = inp.float().to(device)
            out = out.float().to(device)
            
            # 0 pad the end of input seq
            inp = torch.cat((inp, torch.zeros(inp.size(0), 10, inp.size(2), device=device)), dim=1)
            pred = net(inp).to(device)
            # print('input: {}'.format(inp[0, :3]))
            # print('preds: {}'.format(pred[0, :3]))
            # print('true: {}'.format(out[0, :3]))
            
            loss = loss_fct(pred, out)

            criterion.zero_grad()
            loss.backward()
            grad_clipping(net, 1)
            criterion.step()
            
        train_l.append(loss.item())
        
        if valid:
            for i_batch, batch in enumerate(val_loader):
                with torch.no_grad():
                    inp, out = batch
                    inp = inp.float().to(device)
                    out = out.float().to(device)

                    inp = torch.cat((inp, torch.zeros(inp.size(0), 10, inp.size(2), device=device)), dim=1)
                    pred = net(inp).to(device)

                    val_loss = loss_fct(pred, out)
        if valid:
            val_l.append(val_loss.item())
            
            print('epoch: {}, training loss: {}, validation loss: {}'.format(epoch + 1, loss, val_loss))
        else:
            print('epoch: {}, training loss: {}'.format(epoch + 1, loss))
        
    
    fig, ax = plt.subplots(1, 2, figsize=(15, 10))
    sns.lineplot(ax=ax[0], x=np.arange(0, len(train_l)), y=train_l)
    if valid:
        sns.lineplot(ax=ax[1], x=np.arange(0, len(val_l)), y=val_l)
    print('-'* 70)
    return

# def predict(net, data_loader, device):
#     with torch.no_grad():
#         for i_batch, batch in enumerate(data_loader):
#             inp, _ = batch
#             inp = torch.cat((inp.float().to(device), torch.zeros(inp.size(0), 10, inp.size(2)).to(device)), dim=1)
                        
#             preds = net(inp.float().to(device))
            
#             return preds
        
def write_city_preds(net, test_loader, device, city, fp):       
    scene = 0
    output = ''

    with torch.no_grad():
        for i_batch, batch in enumerate(test_loader):
            inp = batch
            inp = inp.float().to(device)
            inp = torch.cat((inp, torch.zeros(inp.size(0), 10, inp.size(2), device=device)), dim=1)

            preds = net(inp)
            flat = preds[0].flatten().cpu().tolist()
            
            row = ['{}_{}'.format(scene, city)] + flat
            row = [str(i) for i in row]
            output += ','.join(row) + '\n'
            
            scene += 1
    
    try:
        with open('./submission.csv', 'a') as f:
            f.write(output)
        print('Predictions for {} generated!'.format(city))
        return 1
    except:
        print('Error! Unsuccessful write...')
        return -1
            
            
            
            

## These models were trained with SGD as the opto and it wasn't very good

In [6]:
# model = RNN(2, 256, 2).to(device)
# opto = torch.optim.SGD(model.parameters(), lr=1)
# loss_fct = nn.MSELoss()
# train(model, 100, train_loader, opto, loss_fct, device)

In [7]:
# model = RNN(2, 128, 2).to(device)
# opto = torch.optim.SGD(model.parameters(), lr=1)
# loss_fct = nn.MSELoss()
# train(model, 100, train_loader, opto, loss_fct, device)

In [8]:
# model = RNN(2, 128, 2).to(device)
# opto = torch.optim.SGD(model.parameters(), lr=.1)
# loss_fct = nn.MSELoss()
# train(model, 100, train_loader, opto, loss_fct, device)

## Started using Adam optimizer / lstm with dropout, these next 2 models are usable, adding l2 loss didnt converge, currently using the second one

In [9]:
# model = RNN(input_size=2, hidden_size=128, output_size=2, num_layers=2, dropout=.2).to(device)
# opto = torch.optim.Adam(model.parameters(), lr=.001)
# loss_fct = nn.MSELoss()

# train(model, 100, train_loader, opto, loss_fct, device)

In [10]:
# model = RNN(input_size=2, hidden_size=256, output_size=2, num_layers=2, dropout=.2).to(device)
# opto = torch.optim.Adam(model.parameters(), lr=.001)
# loss_fct = nn.MSELoss()

# train(model, 100, train_loader, opto, loss_fct, device)

## Bidirectional doesn't make sense here

In [11]:
# model = RNN(2, 256, 2, .2, True).to(device)
# opto = torch.optim.Adam(model.parameters(), lr=.001, weight_decay=.01)
# loss_fct = nn.MSELoss()

# train(model, 100, train_loader, opto, loss_fct, device)

# Test Dataset and Predictions

In [12]:
def generate_submission(fp, model, opto, loss_fct, device, valid=False):
    cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"] 
    header = ['ID'] + ['v' + str(i) for i in range(0, 120)]
    
    with open(fp, 'w') as f:
        f.write(','.join(header) + '\n')

    for city in cities:
        batch_sz = 64
        if valid:
            i, v_i, o, v_o = get_city_trajectories(city=city, split="train", valid=valid)
            training_data = ValidationDataset(i, o)
            validation_data = ValidationDataset(v_i, v_o)
            train_loader = DataLoader(training_data, batch_size=batch_sz)
            val_loader = DataLoader(validation_data, batch_size=batch_sz)
        else:
            val_loader = None
            training_data = ArgoverseDataset(city=city, split='train')
            train_loader = DataLoader(training_data, batch_size=batch_sz)
        
        train(model, 150, train_loader, loss_fct, opto, device, val_loader, valid)
        
        if not valid:
            test_dataset  = ArgoverseDataset(city=city, split='test')
            test_loader = DataLoader(test_dataset, batch_size=1)

            write_city_preds(model, test_loader, device, city, fp)

    print(fp + ' generated!')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RNN(input_size=2, hidden_size=256, output_size=2, device=device, num_layers=2, dropout=.2).to(device)
opto = torch.optim.Adam(model.parameters(), lr=.0005)
loss_fct = nn.MSELoss()

generate_submission('./submission.csv', model, opto, loss_fct, device, valid=False)

epoch: 1, training loss: 1731522.5
epoch: 2, training loss: 1589051.75
epoch: 3, training loss: 1462481.25
epoch: 4, training loss: 1348537.75
epoch: 5, training loss: 1245750.625
epoch: 6, training loss: 1152664.5
epoch: 7, training loss: 1071228.0
epoch: 8, training loss: 998954.75
epoch: 9, training loss: 932759.9375
epoch: 10, training loss: 872911.4375
epoch: 11, training loss: 820262.9375
epoch: 12, training loss: 772294.75
epoch: 13, training loss: 728069.0625
epoch: 14, training loss: 689325.0
epoch: 15, training loss: 654535.5
epoch: 16, training loss: 620733.3125
epoch: 17, training loss: 590793.3125
epoch: 18, training loss: 560846.375
epoch: 19, training loss: 533717.1875
epoch: 20, training loss: 508160.28125
epoch: 21, training loss: 484455.46875
epoch: 22, training loss: 462215.28125
epoch: 23, training loss: 440940.625
epoch: 24, training loss: 418121.25
epoch: 25, training loss: 396001.6875
epoch: 26, training loss: 375554.59375
epoch: 27, training loss: 355326.40625
e